In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle

In [2]:
X  = np.load("X.npy")
Y = np.load("Y.npy")
X,Y = shuffle(X,Y)
print(X.shape)
print(Y.shape)
print(np.shape(X[1,:]))
LOGDIR = r"C:\Users\Vishnu\Documents\EngProj\Kaggle\sign_language\logs"

(2062, 64, 64)
(2062, 10)
(64, 64)


In [3]:
#Functions Tensorflow
def data_input(ipt,labels,b_size,tp):
    with tf.name_scope("data_prep_"+tp):
        dset = tf.data.Dataset.from_tensor_slices((ipt,labels))
        dset = dset.shuffle(max(ipt.shape),reshuffle_each_iteration=True)
        dset = dset.map(map_fun).batch(b_size)
        itr = tf.data.Iterator.from_structure(dset.output_types,dset.output_shapes)
        itr_init_op = itr.make_initializer(dset)
        return itr,itr_init_op
    
def map_fun(ipt,labels):
    #ipt = tf.image.per_image_standardization(ipt)
    return [ipt,labels]
    
def get_weight(shape,name):
    initial_weights = tf.truncated_normal(shape,stddev=0.1)
    return tf.variable(initial_weights,name)

def get_bias(shape,name):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial,name)

def conv_2d(ipt,size_in,size_out,k_size_w,k_size_h,pool_k_size,pool_stride_size,numb):
    with tf.name_scope("Conv"+numb):      
        #Relu Standard Deviation
        std = np.power(2.0/(k_size_w*k_size_h*size_in),.5)
        print("conv"+numb,std)
        #in_w = tf.truncated_normal([k_size_w,k_size_h,size_in,size_out],stddev=std)
        W = tf.Variable(tf.truncated_normal([k_size_w,k_size_h,size_in,size_out],stddev=std),name="W"+numb)
        b = tf.constant(0.1,shape=[size_out],name="B"+numb)
        
        c = tf.nn.conv2d(ipt,W,strides=[1,1,1,1],padding="SAME",name="Conv"+numb)
        act = tf.nn.leaky_relu(c+b,alpha=0.3)
        #act = tf.nn.relu(c+b)
        
        tf.summary.histogram("Weights",W)
        tf.summary.histogram("Bias",W)
        tf.summary.histogram("Activation",act)
        return tf.nn.max_pool(act,ksize=[1,k_size_w,k_size_h,1],strides=[1,pool_stride_size,pool_stride_size,1],padding="SAME",name="pool"+numb)
    
def fc_layer(ipt,size_in,size_out,batch_d,numb):
    with tf.name_scope("FC"+numb):
        #Using Batch Norm
        #Initialize Weights
        std = np.power(2.0/(size_in*size_out),.5)
        print("FC"+numb,std)
        W = tf.Variable(tf.truncated_normal([size_in, size_out],stddev=std),name="W_FC"+numb)
        #b = tf.constant(0.1,shape=[size_out],name="B_FC"+numb)
        z = tf.matmul(ipt,W)
        bt_norm = batch_norm(z,numb,batch_d)
        
        act = tf.nn.leaky_relu(bt_norm,alpha=.1)
        act = tf.nn.dropout(act,tf.get_default_graph().get_tensor_by_name("placeholder/keep_prob:0"))
        
        
        tf.summary.histogram("Weights",W)
        tf.summary.histogram("Activation",act)
        return act
    
def batch_norm(ipt,numb,decay):
    with tf.name_scope("Batch_norm"+numb):
        #Defining Batch Terms
        b_beta = tf.Variable(tf.zeros(ipt.get_shape()[-1]),name="beta"+numb)
        b_gamma= tf.Variable(tf.ones(ipt.get_shape()[-1]),name="gamma"+numb)
        #Population Terms
        pop_mean = tf.Variable(tf.zeros(ipt.get_shape()[-1]),trainable=False,name="pop_mean"+numb)
        pop_var = tf.Variable(tf.ones(ipt.get_shape()[-1]),trainable=False,name="pop_var"+numb)
    
        tf.summary.histogram("Beta",b_beta)
        tf.summary.histogram("Gamma",b_gamma)
        tf.summary.histogram("pop_mean",pop_mean)
        tf.summary.histogram("pop_var",pop_var)
        
        #If is training
        def f_true():
            m,var = tf.nn.moments(ipt,axes=[0],name="comp_mvar"+numb)
            t_mean = tf.assign(pop_mean,pop_mean*decay+m*(1-decay),name="T_mean")
            t_var = tf.assign(pop_var,pop_var*decay+var*(1-decay),name="T_var")
            
            
            
            with tf.control_dependencies([t_mean,t_var]):
                return tf.nn.batch_normalization(ipt,m,var,b_beta,b_gamma,variance_epsilon=.05,name="bnorm"+numb)
        
            
        #If is inference
        def f_false():
            return tf.nn.batch_normalization(ipt,pop_mean,pop_var,b_beta,b_gamma,variance_epsilon=0.5,name="bnorm"+numb)
        
        t = tf.get_default_graph().get_tensor_by_name("placeholder/is_training:0")
        rtn_val = tf.cond(t,f_true,f_false,name="Pop_or_Batch")
        return rtn_val
    

In [4]:
#Model 
#def model(conv_layers,kernels,fc_layers,nodes,learning_rate):
def model(conv_layers,fc_layers,learning_rate,lr_decay,batch_size,batch_d,epochs):
    #Prelims
    tf.reset_default_graph()
    g_step = tf.train.create_global_step()
        
    #Seperating Data
    tot_data = len(X)
    T_data_len = int((2.0/3)*tot_data)
    print(T_data_len)
    TX = X[0:T_data_len,:]
    TY = Y[0:T_data_len,:]
    
    TrX = X[T_data_len+1:,:]
    TrY = Y[T_data_len+1:,:]
    print(len(TX),len(TrX))
    TX = TX[:,:,:,np.newaxis]
    TY = TY[:,:,np.newaxis]
    
    TrX = TrX[:,:,:,np.newaxis]
    TrY = TrY[:,:,np.newaxis]
    print(np.shape(TX),np.shape(TrX))
    #Data Iterators

    T_iter, T_iter_init_op = data_input(TX,TY,batch_size,"Train")
    TR_iter, TR_iter_init_op = data_input(TrX,TrY,batch_size,"Test")
    
    #Placeholder
    with tf.name_scope("placeholder"):
        is_training = tf.placeholder(tf.bool,name="is_training")
        keep_prob = tf.placeholder(tf.float32,name="keep_prob")
        
    #Variables
    with tf.name_scope("Variable"):
        learning_rt = tf.Variable(learning_rate,name="learning_rate")

    with tf.name_scope("lr_decay"):
        dec_lr = tf.train.exponential_decay(learning_rt,g_step,batch_size,lr_decay,staircase=True)
        tf.summary.scalar("Learning_rate",dec_lr)
    
    
    def f_true():
        rtval = T_iter.get_next()
        return rtval
    
    def f_false():
        rtval = TR_iter.get_next()
        return rtval
    
    dat = tf.cond(is_training,f_true,f_false,strict=True,name="Conditional")
    
    img = dat[0]
    label = dat[1]
    img = tf.reshape(img,[-1,64,64,1])
    label = tf.reshape(tf.cast(label,tf.int32),[-1,10])

    
    
    
    #Model Definition
    convs = []
    convs.append(img)
    conv_layers[0] = 1
    for i in range(0,len(conv_layers)-1):
        convs.append(conv_2d(convs[i],conv_layers[i],conv_layers[i+1],5,5,5,2,str(i+1)))
    
    f_convs = convs[len(convs)-1]
    f_convs_shp = f_convs.get_shape().as_list()
    fc_layers[0] = f_convs_shp[1]*f_convs_shp[2]*f_convs_shp[3]
    print("FC_LAYERS",fc_layers[0])
    fcs = []
    print("BF reshape",f_convs.get_shape().as_list())
    flatten = tf.reshape(f_convs,[-1,fc_layers[0]])
    fcs.append(flatten)
    
    for i in range(0,len(fc_layers)-1):
        fcs.append(fc_layer(fcs[i],fc_layers[i],fc_layers[i+1],batch_d,str(i+1)))
    
    
    es = fcs[len(fcs)-1]
    with tf.name_scope("xentropy"):
        xtrpy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=es,labels=tf.stop_gradient(label)))
        tf.summary.scalar("cross_entropy",xtrpy)
        
    with tf.name_scope("Train"):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            #train = tf.train.GradientDescentOptimizer(dec_lr).minimize(xtrpy,global_step = g_step)
            train = tf.train.AdamOptimizer(dec_lr).minimize(xtrpy,global_step = g_step)
    
    with tf.name_scope("Accuracy"):
        pred = tf.equal(tf.argmax(es,1),tf.argmax(label,axis=1))
        accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
        tf.summary.scalar("acc",accuracy)
        
    summ = tf.summary.merge_all()
    writer = tf.summary.FileWriter(LOGDIR)
    cor = 0
    p = 0
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        #sess.run(T_iter_init_op)

        writer.add_graph(sess.graph)

        for i in range(1,epochs):
            ac = []
            sess.run(T_iter_init_op)
            while(True):
                p = p+1
                try:
                    a,acc,s = sess.run([train,accuracy,summ],{is_training: True,keep_prob: 1.0})
                    writer.add_summary(s,p)
                    #print(np.argmax(ess,axis=1),np.argmax(labels,axis=1))
                    #print(tm,tv)
                    ac.append(acc)
                except tf.errors.OutOfRangeError:
                    print("Finished Training: "+str(i))
                    break
            print(np.average(acc))
            
        writer.close()
        
        cor = 0
        sess.run(T_iter_init_op)
        while(True):
            try:
                ess,labels,pp = sess.run([es,label,pred],{is_training: True, keep_prob: 1.0})
                for i in pp:
                    if (i):
                        cor = cor+1
            except tf.errors.OutOfRangeError:
                print("Finished Infer")
                break;
        print("Training Acc: ",float(cor)/max(TX.shape))
        
        
        sess.run(TR_iter_init_op)
        cor = 0
        while(True):
            try:
                ess,labels,pp= sess.run([es,label,pred],{is_training: False,keep_prob:1.0})
                for i in pp:
                    if (i):
                        cor = cor+1
            except tf.errors.OutOfRangeError:
                print("Finished Infer")
                break
        
        
        print("Test Acc: ",float(cor)/len(TrX))

In [5]:
conv = [0,8,16,32]
fclyr = [0,1000,300,150,50,10]
bs = 30
lr = 1.0
lrd = .9
epochs = 15
b_decay = .8
model(conv,fclyr,lr,lrd,bs,b_decay,epochs)

1374
1374 687
(1374, 64, 64, 1) (687, 64, 64, 1)
conv1 0.282842712474619
conv2 0.1
conv3 0.07071067811865475
FC_LAYERS 2048
BF reshape [None, 8, 8, 32]
FC1 0.0009882117688026185
FC2 0.0025819888974716113
FC3 0.006666666666666667
FC4 0.016329931618554522
FC5 0.06324555320336758
Finished Training: 1
0.29166666
Finished Training: 2
0.33333334
Finished Training: 3
0.7083333
Finished Training: 4
0.6666667
Finished Training: 5
0.7083333
Finished Training: 6
0.7083333
Finished Training: 7
0.875
Finished Training: 8
0.9166667
Finished Training: 9
0.875
Finished Training: 10
0.9583333
Finished Training: 11
0.9583333
Finished Training: 12
0.9166667
Finished Training: 13
0.9583333
Finished Training: 14
1.0
Finished Infer
Training Acc:  0.9679767103347889
Finished Infer
Test Acc:  0.9737991266375546
